In [2]:
import base64
import copy
import datetime
import json
import os
import time
from concurrent.futures import ThreadPoolExecutor

import matplotlib.image as mpimg  # mpimg 用于读取图片
import matplotlib.pyplot as plt  # plt 用于显示图片
import numpy as np
import requests
from fake_useragent import UserAgent
from wxpusher import WxPusher

import CoodiSys
from CoodiSys import BOUND_LOCATION
from net_control.push_helper import UIDS, TOPIC_IDS
from net_control.req_misc import input_with_timeout

headers_with_default_UA = {"Accept": "application/json, text/plain, */*", "Accept-Encoding": "br, gzip, deflate",
                           "Accept-Language": "zh-cn",
                           "Connection": "close",
                           "Content-Length": "131",
                           "Content-Type": "text/plain;charset=UTF-8",
                           "Host": "api.ttbike.com.cn",
                           "Origin": "http://m.ttbike.com.cn",
                           "Referer": "http://m.ttbike.com.cn/ebike-h5/latest/index.html",
                           "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 "
                                         "(KHTML, "
                                         "like Gecko) Version/12.0 MQQBrowser/8.8.2 Mobile/16A5345f Safari/604.1 "
                                         "MttCustomUA/2 "
                                         "QBWebViewType/1 WKType/1"}

headers_with_default_UA_new = {
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'content-length': '325',
    'content-type': 'text/plain;charset=UTF-8',
    'origin': 'http://m.ttbike.com.cn',
    'referer': 'http://m.ttbike.com.cn/',
    'requestid': '3BFuzxLoDOwqO4D',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': ''
}

# <editor-fold desc="Data Capture Section">
"120.68976,27.91788 ,120.72077,27.93722"

DEFAULT_COOLDOWN = 0.6
TOKEN_LENGTH = 32
ROOT_FOLDER = 'RecoveredBikeData/'

"""
available_code,phoneNumber,last_phone_used_time,expired_code,LoginToken,last_LoginToken_used_time,CoolDown_time
"""
normal_format_dict = {'PhoneNumber': -1, 'available_code': -1, 'last_phone_used_time': -1.0,
                      'expired_code': -1, 'LoginToken': '', 'last_LoginToken_used_time': -1.0, 'CoolDown_time': -1.0}

normal_format_order = ['PhoneNumber', 'available_code', 'last_phone_used_time',
                       'LoginToken', 'expired_code', 'last_LoginToken_used_time', 'CoolDown_time']


# <editor-fold desc="basic_txt operation">
def create_blank_txt():
    print('local_phone.txt is not found')
    print('creating a blank phone list')
    with open('dataset/local_phone.txt', 'w'):
        pass
    if os.path.exists('dataset/local_phone.txt'):
        print('successfully creating a local_phone.txt')
    else:
        print('fail to create a local_phone.txt')


def cheek_local_phone_format() -> int:
    """
    check text format
    1.create one if None
    2.return BAD LINE serial
    """
    print('cheek local_phone.txt file existence')
    if os.path.exists("../../local_phone.txt"):

        with open("../../local_phone.txt", "r") as f:
            lineCounter = 0
            line = f.readline()
            while line != '':
                line = line.split()
                lineCounter += 1

                print(f'current cheeking line {lineCounter} : {line}')
                if len(line) != len(normal_format_order):
                    print(f'bad params line {lineCounter}\n{line}\n')
                    return lineCounter
                line = f.readline()

        print(f'[{lineCounter}] lines in total, No obvious error')
        return -1
    else:
        create_blank_txt()
        return 0


def sync_phone_txt(add="local_phone.txt"):
    """
    reset local_phone.txt
    record phone number and then create a new local_phone.txt with other data reset to -1
    """
    with open(add, "r") as f:  # record all the phone numbers
        lineCounter = 0
        phone_number_list = []
        line = f.readline()
        while line != '':
            line = line.split()
            lineCounter += 1

            print(f'current OverWriting line {lineCounter} : {line}')

            phone_number_list.append(line[0])  # phone number list assemble
            line = f.readline()

    with open(add, "w") as f:  # draw the records to the text and sync other data
        for phoneNumber in phone_number_list:
            f.write(phoneNumber)
            for ele in range(len(normal_format_order) - 2):
                f.write(' -1')
            f.write(f' {DEFAULT_COOLDOWN}\n')  # default cooldown time


def get_all_lines_count(filenameAdd='local_phone.txt') -> int:
    """
    Get the number of lines in a file
    """
    with open(filenameAdd, 'r') as f:
        lines = f.readlines()
    return len(lines)


def get_lines_with_content_Count(filenameAdd='local_phone.txt') -> int:
    """

    :param filenameAdd:
    :return:
    """
    with open(filenameAdd, 'r') as f:
        lines = f.readlines()
        length = len(lines)
        for line in lines:
            if line == '\n':
                length -= 1
    return length


# </editor-fold>


# <editor-fold desc="HAL">


# <editor-fold desc="net_work_components">
def a_random_header():
    tempHeader = copy.deepcopy(headers_with_default_UA)
    tempHeader['User-Agent'] = UserAgent().random
    return tempHeader.copy()


def sendSMSCode_ttBike(phone):
    """
    功能：输入手机号，模拟完成获取哈喽单车短信验证码的操作。
    1.传入手机号phone
    2.如果顺利，请求响应会包含base64图片验证码，否则显示异常信息
    3.显示图片验证码，弹框要求输入。如果输入的验证码正确，服务器会返回发送短信成功地响应，否则显示异常信息
    """
    secs = requests.session()

    get_code_url = 'https://api.ttbike.com.cn/auth?user.account.sendCodeV2'
    get_code_data = {"version": "4.2.3", "from": "h5", "systemCode": 63, "platform": 6,
                     "action": "user.account.sendCodeV2", "mobile": phone, "capText": ""}
    r = secs.post(get_code_url, headers=a_random_header(), data=json.dumps(get_code_data), timeout=6)  # post手机号
    status = False
    if "imageCaptcha" in r.text:  # 获取短信验证码前需要验证图片验证码，如果response包含验证码字段，则正确，否则异常
        img = json.loads(r.text)["data"]["imageCaptcha"]
        with open("captcha.png", 'wb') as tempOut:
            tempOut.write(base64.decodebytes(bytes(img[22:], "utf-8")))  # 将base64图片验证码解码保存
        captcha = mpimg.imread('captcha.png')  # 使用mpimg读取验证码
        plt.imshow(captcha)  # 在Ipython或JupyterNoetebook显示图片，便于人工输入
        plt.axis('off')
        plt.show()
        code = input('请输入图片验证码:')
        get_code_data["capText"] = code
        r = secs.post(get_code_url, headers=a_random_header(), data=json.dumps(get_code_data), timeout=6)
        if 'true' in r.text:
            print(r.text)
            print('短信验证码发送成功！')
            status = True
        else:
            print('图片验证码错误！')
    else:
        print(r.text)
    return status


def getToken_ttBike(phone, code) -> str:
    """
    功能：输入手机号和短信验证码，模拟用户登陆，获取token
    1.传入手机号和对应的短信验证码
    2.如果登陆成功，返回token，否则显示异常信息
    """
    session = requests.session()

    login_url = 'https://api.ttbike.com.cn/auth?user.account.login'
    login_data = {"version": "4.2.3", "from": "h5", "systemCode": 63, "platform": 1, "action": "user.account.login",
                  "mobile": phone, "code": code, "picCode": {"cityCode": "0577", "city": "温州市", "adCode": "330304"}}

    r = session.post(login_url, headers=a_random_header(), data=json.dumps(login_data), timeout=6)
    print(r.text)
    token = json.loads(r.text)["data"]["token"]
    print(f'获取token成功,the token is: {token}')
    return token


def getBikes_improved(point_coordinates: list, token: str, USE_NEW_VERSION=False) -> list:
    """
    功能：获取某一经纬度周边500(?)米的所有单车信息
    1.传入经纬度和token值
    2.如果顺利，返回经纬度周围500米的所有单车信息，否则显示异常信息
    """

    get_bike_data = {"version": "4.2.3", "from": "h5", "systemCode": 63, "platform": 1,
                     "action": "user.ride.nearBikes",
                     "lng": point_coordinates[0], "lat": point_coordinates[1],
                     "currentLng": point_coordinates[0], "currentLat": point_coordinates[1],
                     "cityCode": "0577", "adCode": "330304", "token": token
                     }

    get_bike_data_new = {
        "version": "6.17.0", "from": "h5", "systemCode": 63, "platform": 1,
        "action": "user.ride.nearBikes",
        "lng": point_coordinates[0], "lat": point_coordinates[1],
        "currentLng": point_coordinates[0], "currentLat": point_coordinates[1],
        "adCode": "330304", "cityCode": "0577", "token": token,
        "ticket": "MTY3MjQxMjQ3Mg==.su9UiPSWh6VGUPHMqk8tpmD3wpgVr8eB464cPOT/J9o="
    }
    if USE_NEW_VERSION:
        get_bike_data = get_bike_data_new

    get_bike_url = 'https://api.ttbike.com.cn/api?user.ride.nearBikes'
    bikeData_return = requests.post(get_bike_url, headers=a_random_header(), data=json.dumps(get_bike_data), timeout=20)
    print(f'Server Echoing status:{bikeData_return}')
    Bike_raw_data_dict = json.loads(bikeData_return.text)  # return type is dict

    """

    passCode = 0
    expiredCode = 133
    """

    if Bike_raw_data_dict['data']:
        return Bike_raw_data_dict['data']  # a list

    else:
        # print(bikeData_return.text)
        print(f'## BAD TOKEN ## : {token}')
        return [token]


# </editor-fold>


def get_SMS_Code_Manually() -> int:
    """
    Get the SMS code
    """
    print('needing human configuration')
    SMScode = int(input_with_timeout('Please input the SMScode: '))
    if SMScode is None:
        print('no response,skip input')
        return 0000
    return SMScode


def load_local_phone_dict_list():
    """
    load local phone dict list
    :return:
    """
    local_phone_dict_list = []

    with open('local_phone.txt', 'r') as f:

        for i in range(get_all_lines_count()):
            line = f.readline()

            if line == '\n' and line == '':
                print(f'skip blank lines')
                continue
            print(f'loading line :{line}')
            tempDict = copy.deepcopy(normal_format_dict)
            line_list = line.split()  # converting strings to list
            for order_place in range(len(normal_format_order)):  # converting list to dict
                order_place_type = type(tempDict[normal_format_order[order_place]])
                tempDict[normal_format_order[order_place]] = order_place_type(line_list[order_place])
            local_phone_dict_list.append(tempDict)
        print('All lines were converted successfully\n###################################\n\n\n')
    return local_phone_dict_list


def write_local_phone_dict_list_to_file(dict_list: list, address='local_phone.txt'):
    """
    write local phone dict list to file
    :param dict_list:
    :param address:
    :return:
    """
    # print(f'writing {dict_list}\n to file {address}\n##############################')
    show_dict_list(dict_list)
    with open(address, 'w') as f:
        for phone_data_dict in dict_list:
            spaceC = ''
            for order_place in normal_format_order:
                f.write(spaceC + str(phone_data_dict[order_place]))
                if spaceC == '':
                    spaceC = ' '
            f.write('\n')
    print('All Written!')


# </editor-fold>


# <editor-fold desc="Function Updates">


def rectangle_slice(loc_list: list = BOUND_LOCATION, step=0.0009,
                    disPlayPic: bool = False) -> list:
    """
    attention: This function will not function properly when called with a line_liked tangle,demanding img improvements
    BOUND_LOCATION: defined in the other docs
    :param disPlayPic:
    :param step:
    :param loc_list: containing two conner coordinates
    :return: node_list containing point within the rectangle defined by two conner

    lat
    |
    |
    |_______lng
    """
    node_list = []
    Node = [0, 0]
    # print(f'{loc_list}')
    latRange = np.arange(loc_list[1], loc_list[3], step)
    lngRange = np.arange(loc_list[0], loc_list[2], step)

    total_nodes_counter = len(lngRange) * len(latRange)
    print(f'the monitoring area contains {total_nodes_counter}')
    for i in range(len(lngRange)):
        for j in range(len(latRange)):
            tempNode = copy.deepcopy(Node)
            tempNode[0] = lngRange[i]  # x
            tempNode[1] = latRange[j]  # y
            node_list.append(tempNode.copy())
    if disPlayPic:
        data_display_multiplier = 1

        plt.figure(figsize=(len(lngRange) / 1.2, len(latRange) / 1.2), dpi=130)

        plt.axis('equal')

        for node in node_list:
            plt.scatter(node[0], node[1], s=5)
            plt.scatter(node[0] * data_display_multiplier, node[1] * data_display_multiplier, alpha=0.5, s=2000)
        plt.grid()
        plt.suptitle(f"({len(lngRange)}X{len(latRange)}) {len(node_list)} Points", x=0.12, y=0.98, fontsize=16,
                     color='red')
        plt.title(datetime.datetime.now().strftime('%Y-%m-%d %H:%M'))
        plt.xlabel(f'Longitude (deg*{data_display_multiplier})', loc='left')
        plt.ylabel(f'Latitude (deg*{data_display_multiplier})', loc='top')
        plt.show()
    return node_list


def multiTangle_slice(node_list: list, loc_list: list, disPlayPic: bool = False):
    """

    :param node_list:
    :param loc_list:
    :param disPlayPic:
    :return:
    """

    pass


def check_point_in_tangle(point: list, tangle: list) -> bool:
    """

    :param point:
    :param tangle:
    :return:
    """
    inRange = False
    if tangle[0] < point[0] < tangle[2]:  # x lock
        if tangle[1] < point[1] < tangle[3]:  # y lock
            inRange = True

    return inRange


def bikeDict_to_location(bikeDict: dict):
    return [float(bikeDict.get('lng')), float(bikeDict.get('lat'))]


def filter_point_in_tangle(bikeList: list, tangle: list) -> list:
    """
    lat
    |
    |
    |_______lng

    :param bikeList:
    :param tangle:
    :return:

    """
    inRangeBikeList = []

    for bikeDict in bikeList:
        bikeLocation = bikeDict_to_location(bikeDict)
        if check_point_in_tangle(bikeLocation, tangle=tangle):
            inRangeBikeList.append(bikeDict)
    return inRangeBikeList


# </editor-fold>


def get_all_bikes(dict_list: list, loc_list: list = BOUND_LOCATION,
                  stepLen: float = 0.0008, USE_NEW_VERSION: bool = False) -> list:
    """
    no judge if the data obtained is in range
    :param USE_NEW_VERSION:
    :param stepLen:
    :param dict_list:
    :param loc_list:
    :return:
    """
    allBikes = []  # store result

    points_list = rectangle_slice(loc_list=loc_list, step=stepLen)  # area slice

    token_s_dict = loop_find_available_token(dict_list)
    points_list_len = len(points_list)

    extractStartTime = datetime.datetime.now()  # time Consumptions calc

    point_serial = 0
    for point in points_list:
        startFrame = time.time()
        # print(f'Going with Point {point_serial}/{points_list_len}: {point}........')    #for debugging purposes

        bike_raw_list = getBikes_improved(point, token_s_dict[normal_format_order[3]],
                                          USE_NEW_VERSION=USE_NEW_VERSION)

        if bike_raw_list[0] == token_s_dict[normal_format_order[3]]:  # return equal token suggesting fail getBikes

            update_token_status(token_s_dict, expired_code=1)  # update bad token status

            updateResult = '## FAIL ##'

        else:
            allBikes.extend(bike_raw_list)
            update_token_status(token_s_dict, token=token_s_dict[normal_format_order[3]])
            updateResult = 'Success'

        print(
            f'point {point_serial}/{points_list_len}: {point} ,{updateResult},using {(time.time() - startFrame):.4f}s\r\n'
            f'--------------------------------')
        point_serial += 1
        token_s_dict = loop_find_available_token(dict_list)
    livingTIme = datetime.datetime.now() - extractStartTime
    print(f'#################################\n'
          f'Consumed time: {livingTIme.seconds}s\n'
          f'Average time per point : {(livingTIme.seconds / points_list_len):.3f}s'
          f'\n#################################\n')

    return allBikes


def get_all_bikes_MT(dict_list: list, loc_list: list = BOUND_LOCATION, stepLen: float = 0.0008) -> list:
    """
        no judge if the data obtained is in range
      :param stepLen:
      :param dict_list:
      :param loc_list:
      :return:
      """
    allBikes = []  # store result

    points_list = rectangle_slice(loc_list=loc_list, step=stepLen)  # area slice

    token_s_dict = loop_find_available_token(dict_list)
    points_list_len = len(points_list)

    extractStartTime = datetime.datetime.now()  # time Consumptions calc

    point_serial = 0
    startFrame = time.time()
    with ThreadPoolExecutor(max_workers=len(dict_list)) as pool:  # divide the missions up
        '''
        this is a list is that contains all the point viewed bikes
        '''
        bike_raw_list_pieces = list(pool.map(getBikes_improved, points_list, loop_find_available_token(dict_list)))
        print(bike_raw_list_pieces)
    for list_pieces in bike_raw_list_pieces:

        if list_pieces[0] == token_s_dict[normal_format_order[3]]:  # return equal token suggesting fail getBikes

            update_token_status(token_s_dict, expired_code=1)  # update bad token status

            updateResult = '## FAIL ##'

        else:
            allBikes.extend(list_pieces)
            update_token_status(token_s_dict, token=token_s_dict[normal_format_order[3]])
            updateResult = 'Success'

        print(
            f'point {point_serial}/{points_list_len}: {points_list[point_serial]} \r\n'
            f'{updateResult}\r\n'
            f'using {(time.time() - startFrame):.4f}s\r\n'
            f'--------------------------------')
        point_serial += 1
        token_s_dict = loop_find_available_token(dict_list)
    livingTIme = datetime.datetime.now() - extractStartTime
    print(f'#################################\n'
          f'Consumed time: {livingTIme.seconds}s\n'
          f'Average time per point : {livingTIme.seconds / points_list_len}s'
          f'\n#################################\n')

    return allBikes


def get_all_bikes_Astar(dict_list: list, loc_list: list = BOUND_LOCATION, stepLen: float = 0.0008) -> list:
    """
    no judge if the data obtained is in range
    :param dict_list:
    :param loc_list:
    :param stepLen:
    :return:
    """
    allBikes = []

    init_point_list = rectangle_slice(loc_list=loc_list)

    return allBikes


def open_CurTime_tree_folder(rootFolder: str = ''):
    """

    :param rootFolder:
    :return: the dir that creates
    """
    if rootFolder == '':
        rootFolder = os.path.abspath('../..')
    time_str = time.strftime('%Y-%m %d', time.localtime())
    firstFolder = time_str.split()[0]
    secondFolder = time_str.split()[1]
    created_folder = f'{rootFolder}/{firstFolder}/{secondFolder}/'
    if os.path.exists(created_folder):
        pass
    else:
        os.makedirs(created_folder)
    if os.path.exists(created_folder):

        print('timeTreeFolder has been created')
        return created_folder
    else:
        raise


def writeRes(allBikes, timestamp, storage_dir=ROOT_FOLDER, return_details=False):
    """
    功能：将单车数据追加写入文本文件存储,自动去重
    :param return_details:
    :param storage_dir:
    :param allBikes:
    :param timestamp:
    :return:
    """
    dataFileName = 'allBikes.txt'

    fine_bike_dict_list = []
    duplicated_bike_dict_list = []
    for bike_data_dict_temp in allBikes:

        bike_data_dict = {'bikeNo': bike_data_dict_temp.get('bikeNo'),
                          'lat': bike_data_dict_temp.get('lat'),
                          'lng': bike_data_dict_temp.get('lng')}
        if bike_data_dict in fine_bike_dict_list:
            duplicated_bike_dict_list.append(bike_data_dict)

        else:
            fine_bike_dict_list.append(bike_data_dict)

    print(f'#############################\n'
          f'Duplicated and removed Bikes: {len(duplicated_bike_dict_list)}\n'
          f'#############################')
    # print(duplicated_bike_dict_list)
    print(f'#############################\n'
          f'Detected bikes: {len(fine_bike_dict_list)}\n'
          f'#############################')
    # print(fine_bike_dict_list)
    with open(open_CurTime_tree_folder(rootFolder=storage_dir) + dataFileName, 'a+') as f:
        for bike_data_dict in fine_bike_dict_list:
            f.write(str(timestamp) + '\t')
            f.write(('\t'.join([bike_data_dict['bikeNo'], bike_data_dict['lat'], bike_data_dict['lng']])))
            f.write('\n')
        print(timestamp + 'write done.')
    if return_details:
        return [len(fine_bike_dict_list), len(duplicated_bike_dict_list)]


# <editor-fold desc="MAIN method">
def run_every_other_interval(dict_list: list, step: float = 0.0008, lastingDays: float = 1,
                             loc_list: list = BOUND_LOCATION, notifications=False, filter_ON=False,
                             USE_NEW_VERSION: bool = False, WriteDownLog=False, dispPlayData=False):
    """
    功能：每隔两分钟获取一次高教园的所有单车信息并保存。

    """
    log_file_name = 'bikes_variant.log'
    max_interval = 360
    min_interval = 50

    endDay = datetime.datetime.now() + datetime.timedelta(days=lastingDays)
    while True:
        try:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
            print('########################################################\n'
                  '##########################START#########################\n'
                  '########################################################')
            nextTime = (datetime.datetime.now() + datetime.timedelta(minutes=3))
            print(timestamp)

            allBikes = get_all_bikes(dict_list, stepLen=step,
                                     loc_list=loc_list,
                                     USE_NEW_VERSION=USE_NEW_VERSION)  # return list of all bikes with dict format

            # <editor-fold desc="Filter">
            if filter_ON:

                filtered_bikes = filter_point_in_tangle(allBikes, tangle=loc_list)
            else:
                filtered_bikes = allBikes
            # </editor-fold>

            bike_count_detail = [0] * 2
            bike_count_detail = writeRes(filtered_bikes, timestamp, return_details=True)
            if dispPlayData:
                print(f'Proceeding with data on map')
                display_bikes_on_map(allBikes, loc_list)

            # <editor-fold desc="BIKES counter log">
            if WriteDownLog:  # params to control the logging
                log_dir = open_CurTime_tree_folder(rootFolder=ROOT_FOLDER) + log_file_name
                log_content = {
                    'totalDetectedBikes_within': bike_count_detail[0],
                    'duplicatedBikes': bike_count_detail[1],
                    'totalDetectedBikes': bike_count_detail[0] + bike_count_detail[1],
                    'timestamp': timestamp
                }

                if not os.path.exists(log_dir):  # check if the log file exists
                    with open(log_dir, mode='w'):
                        pass
                with open(log_dir, mode='a') as log:
                    for content_ele in log_content.keys():
                        log.write(f'{log_content[content_ele]}\t')
                    log.write('\n')

            # </editor-fold>

            # <editor-fold desc="Wx commit sec">
            if notifications:
                WxPusher.send_message(f'NormalFormatDBikes: {bike_count_detail[0]}\n'
                                      f'DuplicatedBikes: {bike_count_detail[1]}\n'
                                      f'ALL AREA TotalDetected: {bike_count_detail[0] + bike_count_detail[1]}\n'


                                      f'timestamp: {timestamp}\n'
                                      f'NextCheckTime: {(nextTime - datetime.datetime.now()).seconds}s',
                                      uids=UIDS,
                                      topic_ids=TOPIC_IDS,
                                      token='AT_7gneRS02ois8YkgVWeCS0Q9iEV3Lq4Xl')
            # </editor-fold>

            restTime = (nextTime - datetime.datetime.now()).seconds
            if min_interval < restTime < max_interval:  # sleep time is always smaller than 1800 seconds
                pass
            else:
                restTime = (min_interval + max_interval) / 2
            print(f'Next scanTime is: ' + nextTime.strftime("%Y-%m-%d %H:%M") + f'## Resting {restTime}s')
            time.sleep(restTime)

            if datetime.datetime.now() > endDay:
                print('time up')
                break
        except KeyboardInterrupt:
            # write_local_phone_dict_list_to_file(dict_list)
            print('Program EXITED')
            break


def run_single(dict_list: list, interval: float = 0.0008, lastingDays: float = 1, loc_list: list = BOUND_LOCATION):
    try:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

        print(timestamp)

        allBikes = get_all_bikes_MT(dict_list, stepLen=interval, loc_list=loc_list)
        writeRes(allBikes, timestamp)

    except KeyboardInterrupt:
        print('Program EXITED')
        return


# </editor-fold>

# <editor-fold desc="TEST section">


def getTokenTest(test_times=1):
    for i in range(test_times):
        phone = input('请输入手机号：')
        sendSMSCode_ttBike(phone)
        code = input('请输入收到的验证码：')
        token = getToken_ttBike(phone, code)
        print(f'The token of [{phone}] is :  {token}\n############################')


def show_dict_list(dict_list: list) -> None:
    print('################################################################')
    for a_dict in dict_list:
        print(a_dict)
    print('################################################################\n')
    return


def display_bikes_on_map(bikes_list: list, Area: list) -> None:
    """
    display_bikes_on_map
    :param bikes_list: dict*n
    :param Area: float*2
    :return:

        lat
    |
    |
    |_______lng
    """
    data_display_multiplier = 1
    plt.figure(figsize=(Area[0], Area[1]), dpi=80)
    # plt.grid(True)
    for bike in bikes_list:
        node = [bike['lng'], bike['lat']]

        plt.scatter((node[0] * data_display_multiplier), node[1] * data_display_multiplier, alpha=0.8, s=200)

    plt.title(datetime.datetime.now().strftime('%Y-%m-%d %H:%M'))
    plt.xlabel(f'Longitude (deg*{data_display_multiplier})', loc='left')
    plt.ylabel(f'Latitude (deg*{data_display_multiplier})', loc='top')
    plt.show()

    return


# </editor-fold>

def token_expired_check_timecheck(phone_data_dict: dict):
    """
    Check if phone data is expired
    """
    # 24 hr=86400 sed
    expire_time = 24 * 60 * 60

    if type(phone_data_dict[normal_format_order[3]]) != str:
        phone_data_dict[normal_format_order[4]] = 0  # no token, no expires
        return False
    elif time.time() - phone_data_dict[normal_format_order[2]] > expire_time:
        # token generated time should be the same as the last_phone_used_time
        print('a token has expired, please refresh')

        '''
        since dont know the exact expired_time so the line below is commented out
        '''
        # phone_data_dict[normal_format_order[4]]=1 #exceeds expired_time
        return True

    return False


def check_all_tokens_expires(dict_list: list):
    """
    Check if all tokens have expired

    :param dict_list:
    :return: expired_token_counter
    """
    expired_token_counter = 0
    for data_dict in dict_list:
        expired_token_counter += token_expired_check_timecheck(data_dict)
    return expired_token_counter


def check_token_available(data_dict: dict):
    passedTime = time.time() - data_dict[normal_format_order[5]]

    # breakpoint()
    if passedTime > data_dict[normal_format_order[6]]:

        print(f'{data_dict[normal_format_order[0]]}: TOKEN available|| Passed time: {passedTime:.3f}s')
        return True
    else:
        # print(f'{data_dict[normal_format_order[0]]}: token not available')

        return False


def check_token_usable_with_net(token: str, test_point: list = (120.68976, 27.91788)) -> bool:
    """
    Check if the token is usable with the net
    :param token:
    :param test_point:
    :return:
    """
    if getBikes_improved(test_point, token=token)[0] == token:
        print(f'BAD TOKEN!!,the {token} is NOT usable')
        return False
    else:
        print(f'OK,the {token} is usable')

        return True


def find_available_token(dict_list: list):
    """
    find_available_token
    :return:
    """
    for a_dict in dict_list:

        if check_token_available(a_dict):
            token = a_dict[normal_format_order[3]]
            # print(f'Found an available token:{token}')
            return token
    # print('currently no token is available')
    return False


def loop_find_available_token(dict_list: list, failCounterON=False) -> dict:
    """
    operate on original data and return the token s dict
    :param dict_list:
    :param failCounterON:
    :return: token s dict with origin data use info
    """
    fail_count = 0
    while True:
        for dict_serial in range(len(dict_list)):

            if check_token_available(dict_list[dict_serial]):
                token = dict_list[dict_serial][normal_format_order[3]]
                print(f'Found an available token : {token}')
                update_token_status(dict_list[dict_serial], token, expired_code=0)
                # write_local_phone_dict_list_to_file(dict_list)
                return dict_list[dict_serial]
            else:
                fail_count += 1
                print(f'##fail to find a token for {fail_count} times##', end='\r')
                time.sleep(0.01)

        if failCounterON and fail_count >= 5 * get_lines_with_content_Count():
            print(f'fail_count exceeds limitation'
                  f'\n############################'
                  f'\n############################')
            massive_Update_phone_data_dict_list(dict_list)
            break

        # normal_sleep(DEFAULT_COOLDOWN / 3)


def loop_random_find_available_token(dict_list: list, max_loop_time=500) -> dict:
    startTime = time.time()
    while True:
        if time.time() - startTime > max_loop_time:
            raise Exception("Time out of loop")

        a_random_int = np.random.randint(0, len(dict_list))  # if usable ,returns
        if check_token_available(dict_list[a_random_int]):
            return dict_list[a_random_int]


def update_token_status(phone_data_dict: dict, token: str = '', expired_code=0):
    phone_data_dict[normal_format_order[3]] = token
    phone_data_dict[normal_format_order[4]] = expired_code
    phone_data_dict[normal_format_order[5]] = time.time()
    return phone_data_dict


def update_phone_status(phone_data_dict: dict, available_code):
    phone_data_dict[normal_format_order[1]] = available_code
    phone_data_dict[normal_format_order[2]] = time.time()
    return phone_data_dict


def update_signal_data_dict(a_data_dict: dict, withTokenTest=True):
    phoneNumber = a_data_dict[normal_format_order[0]]
    print(f'\tUpdating phone {phoneNumber} data')
    sendSMSCode_ttBike(phoneNumber)  # matching SMScode with phoneNumber
    print('\tSending SMS Msg')
    update_phone_status(a_data_dict, available_code=0)
    print('\tUpdating phone status')
    SMScode_input = get_SMS_Code_Manually()
    print(f'the input code is {SMScode_input}\n'
          f'###################################')

    if SMScode_input != 0:
        print('\tGetting token form the server')
        token = getToken_ttBike(phoneNumber, SMScode_input)

        print('\tUpdating token status')
        update_token_status(a_data_dict, token)
        a_data_dict[normal_format_order[3]] = token
        return a_data_dict
    else:
        print(f'bad input code: {SMScode_input}')
        raise


def massive_Update_phone_data_dict_list(dict_list: list, safeUpdate=True) -> list:
    """Update phone data dict list"""
    if safeUpdate and cheek_local_phone_format() > 0:
        print('Sync local cache')
        sync_phone_txt()

    for a_data_dict in dict_list.copy():
        update_signal_data_dict(a_data_dict)

    print('###########\n'
          'ALL Updated\n'
          '###########')
    show_dict_list(dict_list)
    return dict_list


def token_echo_test(test_token, test_point=(BOUND_LOCATION[0], BOUND_LOCATION[1])):
    if test_token is None:
        test_token = input(f'input token')
    print(f'token is {test_token}')
    if test_point is None:
        point_str = input('input point:')
        test_point = point_str.split(',')
        print(f'{test_point}')
    Bikes = list(getBikes_improved(test_point, test_token))
    print(f'###################################\n{Bikes[0]}\n###########################################\n')
    print(f'the total count of detect {len(Bikes)}')
    return Bikes


# </editor-fold>


if __name__ == "__main__":
    # <editor-fold desc="FILE pre-check and load phones to memory">
    if cheek_local_phone_format() > 0:
        sync_phone_txt()
    a_origin_list = load_local_phone_dict_list()
    show_dict_list(a_origin_list)
    # </editor-fold>

    TEST_POINT = (120.695064, 27.916269)  # our main teaching building

    # <editor-fold desc="Phone Data update">
    # update_signal_data_dict(a_data_dict=a_origin_list[3])
    # show_dict_list(a_origin_list)
    # write_local_phone_dict_list_to_file(a_origin_list)
    # massive_Update_phone_data_dict_list(a_origin_list)
    # </editor-fold>

    # <editor-fold desc="find radius">
    # for eachToken in a_origin_list:
    #     bike_data_dict_list = token_echo_test(eachToken[normal_format_order[3]], test_point=TEST_POINT)

    # print(bike_data_dict_list[0])
    # distant_list = []
    # for a_dict in a_origin_list:
    #     distant_list.append(a_dict['distance'])
    # print(distant_list)
    # print(f'the length of distant_list is {len(distant_list)}')
    # print(f'the max distance is {max(distant_list)}')
    # </editor-fold>

    # <editor-fold desc="MAIN Section">

    step = 0.00011
    target_Zone = CoodiSys.D_PARK

    rectangle_slice(target_Zone, step=step, disPlayPic=True)  # s

    run_every_other_interval(a_origin_list, loc_list=target_Zone, step=step, notifications=True, filter_ON=True,
                             USE_NEW_VERSION=False, WriteDownLog=True, dispPlayData=False)

    # </editor-fold>


ImportError: cannot import name 'UIDS' from 'net_control.push_helper' (L:\pycharm projects\Bike_Scrapper\net_control\push_helper.py)